<a href="https://colab.research.google.com/github/gingerchien/QuantHub/blob/main/vectorbt_for_beginners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a compilation of **Chad Thackray's Vectorbt for beginners - Full Python Course** located below and a few of my ideas after learning the course:

https://www.youtube.com/watch?v=JOdEZMcvyac&list=PLKCjdQRzJEHzUc09EhgHD_to93yLnjT0P


#What is Vectorbt?

Accodring to their webstite: https://vectorbt.dev/
vectorbt is a Python package for quantitative analysis that takes a novel approach to backtesting: it operates entirely on pandas and NumPy objects, and is accelerated by Numba to analyze any data at speed and scale. This allows for testing of many thousands of strategies in seconds.

In contrast to other backtesters, vectorbt represents complex data as (structured) NumPy arrays. This enables superfast computation using vectorized operations with NumPy and non-vectorized but dynamically compiled operations with Numba. It also integrates Plotly and Jupyter Widgets to display complex charts and dashboards akin to Tableau right in the Jupyter notebook. Due to high performance, vectorbt can process large amounts of data even without GPU and parallelization and enables the user to interact with data-hungry widgets without significant delays.

# 0. Quickstart

* Documentation: https://vectorbt.dev/api/indicators/basic/
* Github: https://github.com/polakowo/vectorbt

In [119]:
#check python version
!python3 --version

Python 3.10.12


## Install Libraries

In [120]:
# ##install vectorbt
# !pip3 install vectorbt

# ##install Yahoo Finance
# !pip3 install yfinance

In [121]:
import vectorbt as vbt
import yfinance as yf
import datetime

## Download Data

In [122]:
end_date = datetime.datetime.now()
start_date= end_date - datetime.timedelta(days=7)
print(start_date)

#btc_price = vbt.YFData.download(['BTC-USD', 'ETH-USD'], interval='1m', start=start_date, end=end_date ,missing_index='drop').get('Close')
btc_price = vbt.YFData.download('BTC-USD', missing_index='drop').get('Close')

2023-12-14 23:37:17.847817


In [123]:
print(btc_price.tail())
print(btc_price.head())
print(type(btc_price))

Date
2023-12-17 00:00:00+00:00    41364.664062
2023-12-18 00:00:00+00:00    42623.539062
2023-12-19 00:00:00+00:00    42270.527344
2023-12-20 00:00:00+00:00    43652.250000
2023-12-21 00:00:00+00:00    43864.332031
Freq: D, Name: Close, dtype: float64
Date
2014-09-17 00:00:00+00:00    457.334015
2014-09-18 00:00:00+00:00    424.440002
2014-09-19 00:00:00+00:00    394.795990
2014-09-20 00:00:00+00:00    408.903992
2014-09-21 00:00:00+00:00    398.821014
Freq: D, Name: Close, dtype: float64
<class 'pandas.core.series.Series'>


## Technical Indicators

In [124]:
#rsi = vbt.RSI.run(btc_price, window=[14,21]) different windows
rsi = vbt.RSI.run(btc_price, window=14)
print(rsi.rsi) #to extract the value, call rsi.rsi

Date
2014-09-17 00:00:00+00:00          NaN
2014-09-18 00:00:00+00:00          NaN
2014-09-19 00:00:00+00:00          NaN
2014-09-20 00:00:00+00:00          NaN
2014-09-21 00:00:00+00:00          NaN
                               ...    
2023-12-17 00:00:00+00:00    55.392152
2023-12-18 00:00:00+00:00    52.656277
2023-12-19 00:00:00+00:00    41.267414
2023-12-20 00:00:00+00:00    49.587286
2023-12-21 00:00:00+00:00    52.558944
Freq: D, Name: (14, Close), Length: 3383, dtype: float64


### Turning Indicator into True/Falst Signals for processing with Vectorbt

In [125]:
entries = rsi.rsi_crossed_below(28)
#print(entries.to_string())

In [126]:
exits = rsi.rsi_crossed_above(80)
#print(exits.to_string())

In [127]:
#test with portfolio pf
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)

#check what's going on
print(pf.stats())

Start                         2014-09-17 00:00:00+00:00
End                           2023-12-21 00:00:00+00:00
Period                               3383 days 00:00:00
Start Value                                       100.0
End Value                                    186.256571
Total Return [%]                              86.256571
Benchmark Return [%]                        9491.311952
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              84.277674
Max Drawdown Duration                2175 days 00:00:00
Total Trades                                         26
Total Closed Trades                                  26
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  61.538462
Best Trade [%]                               101.299088
Worst Trade [%]                               -5

In [128]:
print(pf.total_return()) #can extract individual items from stats()

0.8625657092546052


In [129]:
#plotting results, the most simple plot in vectorbt
pf.plot().show()

# 1. Custom Indicators using the Indicator Factory
* Reference: https://vectorbt.dev/api/indicators/factory/#naive-approach

In [130]:
end_time = datetime.datetime.now()
start_time= end_time - datetime.timedelta(days=3)
btc_price = vbt.YFData.download('BTC-USD', missing_index='drop', start= start_time, end=end_time, interval='1m').get('Close')

In [131]:
print(btc_price)

Datetime
2023-12-18 23:38:00+00:00    42643.496094
2023-12-18 23:39:00+00:00    42627.273438
2023-12-18 23:40:00+00:00    42624.250000
2023-12-18 23:41:00+00:00    42590.000000
2023-12-18 23:42:00+00:00    42605.515625
                                 ...     
2023-12-21 23:31:00+00:00    43915.203125
2023-12-21 23:32:00+00:00    43877.027344
2023-12-21 23:33:00+00:00    43870.878906
2023-12-21 23:34:00+00:00    43855.898438
2023-12-21 23:35:00+00:00    43870.332031
Name: Close, Length: 4088, dtype: float64


## Define Custom Indicator

In [132]:
import numpy as np
def custom_indicator(close, rsi_window=8, ma_window=13):
  rsi = vbt.RSI.run(close, window=rsi_window).rsi.to_numpy() #the .rsi grabs the actual values to turn it into signal later as raw values are not helpful
  ma = vbt.MA.run(close, window=ma_window).ma.to_numpy()#converting to numpy arrays
  #print(rsi)
  #print(ma)
  #create signal
  trend = np.where(rsi>80, -1, 0)
  trend = np.where((rsi<30) & (close<ma) , 1, trend) #only use & version here, AND does not work, for or use |
  #print(trend)
  return trend

#create indicator object so Vectorbt can recognize it for any custom indicators
indicator = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'ma_window'],
    output_names=['value']
).from_apply_func(custom_indicator, rsi_window=5, ma_window=21) #provide default values then point it to the custom indictor

#run and get indicator values
res = indicator.run(btc_price, rsi_window=5, ma_window=8)
#print(res.value)
entries = res.value == 1.0
exits = res.value == -1.0

# https://vectorbt.dev/api/portfolio/base/#custom-metrics
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, seed=42, freq='m')
print(pf.stats()) #if vectorbt could not parse the frequency of the close, it will not return any duration in time units or metrics that requires annualization and throw a bunch of warnings

Start                         2023-12-18 23:38:00+00:00
End                           2023-12-21 23:35:00+00:00
Period                                  2 days 20:08:00
Start Value                                       100.0
End Value                                     97.330216
Total Return [%]                              -2.669784
Benchmark Return [%]                           2.876959
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               4.910141
Max Drawdown Duration                   2 days 08:41:00
Total Trades                                        157
Total Closed Trades                                 156
Total Open Trades                                     1
Open Trade PnL                                -0.014854
Win Rate [%]                                  59.615385
Best Trade [%]                                 0.413607
Worst Trade [%]                               -1

In [133]:
pf.plot().show()

## What if we want a 5 min indicator on the 1 min data?

In [134]:
import numpy as np

end_time = datetime.datetime.now()
start_time= end_time - datetime.timedelta(days=3)
btc_price = vbt.YFData.download(['BTC-USD', 'ETH-USD'], missing_index='drop', start= start_time, end=end_time, interval='1m').get('Close')

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.align.html
# broadcast_axis = 0 , interchangeable w/ 'index', join = 'right' (takes the keys from the right table only (close is the right table, rsi is the left table))
def custom_indicator2(close, rsi_window=14, ma_window=50):
  close_5m = close.resample('5T').last() #converting 1m candles to 5m
  rsi = vbt.RSI.run(close_5m, window=rsi_window).rsi
  rsi, _ = rsi.align(close, broadcast_axis=0, method='ffill', join='right') #this makes the rsi align to the 1 min but at 5 min intervals, where the other 4 min are NAN in between. forward fill helps fill in the gap.

  #convert everything to numpy
  close = close.to_numpy()
  rsi = rsi.to_numpy()

  ma = vbt.MA.run(close, window=ma_window).ma.to_numpy()
  #print(rsi)
  #print(ma)
  #create signal
  trend = np.where(rsi>70, -1, 0)
  trend = np.where((rsi<30) & (close<ma) , 1, trend) #only use & version here, AND does not work, for or use |
  #print(trend)
  return trend

#create indicator object so Vectorbt can recognize it for any custom indicators
indicator2 = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'ma_window'],
    output_names=['value']
).from_apply_func(custom_indicator2, rsi_window=14, ma_window=21, keep_pd=True) #keep_pd = True, this ensures everything stays in panda format instead of getting turned into numpy arrays

#run and get indicator values
res = indicator2.run(btc_price, rsi_window=5, ma_window=21)
#print(res.value)
entries = res.value == 1.0
exits = res.value == -1.0

# https://vectorbt.dev/api/portfolio/base/#custom-metrics
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, seed=42, freq='m')
print(pf.stats()) #if vectorbt could not parse the frequency of the close, it will not return any duration in time units or metrics that requires annualization and throw a bunch of warnings

/usr/local/lib/python3.10/dist-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Dropping missing data points.



Start                         2023-12-18 23:38:00+00:00
End                           2023-12-21 23:34:00+00:00
Period                                  2 days 18:02:00
Start Value                                       100.0
End Value                                     94.118461
Total Return [%]                              -5.881539
Benchmark Return [%]                           1.847761
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               7.071691
Max Drawdown Duration                   2 days 16:57:00
Total Trades                                       40.0
Total Closed Trades                                39.5
Total Open Trades                                   0.5
Open Trade PnL                                -0.194013
Win Rate [%]                                  48.108974
Best Trade [%]                                 0.545902
Worst Trade [%]                               -1

<ipython-input-134-73c46a6011d0>:44: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x79a4f45d5cf0>. Pass column to select a single column/group.



# 2. Hyperparameter Optimization

In [135]:
end_time = datetime.datetime.now()
start_time= end_time - datetime.timedelta(days=3)
btc_price = vbt.YFData.download(['BTC-USD', 'ETH-USD'], missing_index='drop', start= start_time, end=end_time, interval='1m').get('Close')

def custom_indicator2(close, rsi_window=14, ma_window=50, entry = 30, exit=70):
  close_5m = close.resample('5T').last()
  rsi = vbt.RSI.run(close_5m, window=rsi_window).rsi
  rsi, _ = rsi.align(close, broadcast_axis=0, method='ffill', join='right')

  #convert everything to numpy
  close = close.to_numpy()
  rsi = rsi.to_numpy()

  ma = vbt.MA.run(close, window=ma_window).ma.to_numpy()
  #print(rsi)
  #print(ma)
  #create signal
  trend = np.where(rsi>exit, -1, 0)
  trend = np.where((rsi<entry) & (close<ma) , 1, trend)
  #print(trend)
  return trend

#create indicator object so Vectorbt can recognize it for any custom indicators
indicator2 = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'ma_window', 'entry', 'exit'],
    output_names=['value']
).from_apply_func(custom_indicator2, rsi_window=14, ma_window=50, entry=30, exit=70, keep_pd=True)

#run and get indicator values
res = indicator2.run(btc_price, rsi_window=np.arange(2,15,step=1, dtype=int), ma_window=np.arange(2,13,step=1, dtype=int), entry=np.arange(13,21,step=1, dtype=int),exit=np.arange(60,85,step=4, dtype=int), param_product=True) #param_product = True shows a combination of all the parameters

entries = res.value == 1.0
exits = res.value == -1.0

# https://vectorbt.dev/api/portfolio/base/#custom-metrics
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, seed=42, freq='m')
returns = pf.total_return()#print out everything by using .to_string()
print(returns.max(), returns.idxmax()) #print out the max combination of returns

print(pf.stats()) #if vectorbt could not parse the frequency of the close, it will not return any duration in time units or metrics that requires annualization and throw a bunch of warnings

/usr/local/lib/python3.10/dist-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Dropping missing data points.



0.03559893574628731 (8, 7, 16, 72, 'BTC-USD')
Start                         2023-12-18 23:38:00+00:00
End                           2023-12-21 23:34:00+00:00
Period                                  2 days 18:02:00
Start Value                                       100.0
End Value                                      96.62899
Total Return [%]                               -3.37101
Benchmark Return [%]                           1.847761
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                                4.93249
Max Drawdown Duration         2 days 05:19:31.277472527
Total Trades                                  24.171891
Total Closed Trades                           23.915022
Total Open Trades                              0.256868
Open Trade PnL                                -0.068731
Win Rate [%]                                  55.041325
Best Trade [%]                                  0.49383
Wo

<ipython-input-135-549d6a652db2>:43: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x79a4f45d5cf0>. Pass column to select a single column/group.



## To show the other columns, call it out specifically

In [136]:
# print(returns.to_string()) #print out everything by using .to_string()
# returns = returns[returns.index.isin(['ETH-USD'], level='symbol')]

# print(returns.max(), returns.idxmax()) #print out the max combination of returns
# #print(returns.to_string()) #print out everything by using .to_string()
# print(pf.stats()) #if vectorbt could not parse the frequency of the close, it will not return any duration in time units or metrics that requires annualization and throw a bunch of warnings

## Creating Heatmaps

In [137]:
#plotly is used in vectorbt, x_level and y_level can be any parameters
fig = returns.vbt.heatmap(x_level='comb_rsi_window', y_level='comb_entry',
                          slider_level='comb_ma_window')#adding the slider, can use it to sort symbol best combo
fig.show()

## Groupby Axis to Aggregate Parameters if A Lot of Parameters

In [138]:
#Give us a heatmap grouped by the exit and entry values, mean as aggregation function
#for each entry and exit path, it will average the result of parameters
#This will give us only 3 columns and everything else is averaged out
returns = returns.groupby(level=['comb_exit', 'comb_entry', 'symbol']).mean()
print(returns.to_string())
print(returns.max())
print(returns.idxmax())

comb_exit  comb_entry  symbol 
60         13          BTC-USD   -0.022401
                       ETH-USD   -0.041289
           14          BTC-USD   -0.022366
                       ETH-USD   -0.045229
           15          BTC-USD   -0.022265
                       ETH-USD   -0.044586
           16          BTC-USD   -0.023184
                       ETH-USD   -0.045499
           17          BTC-USD   -0.025521
                       ETH-USD   -0.046277
           18          BTC-USD   -0.026605
                       ETH-USD   -0.051896
           19          BTC-USD   -0.026705
                       ETH-USD   -0.055328
           20          BTC-USD   -0.027689
                       ETH-USD   -0.058692
64         13          BTC-USD   -0.021175
                       ETH-USD   -0.039633
           14          BTC-USD   -0.021506
                       ETH-USD   -0.043376
           15          BTC-USD   -0.021101
                       ETH-USD   -0.042683
           16          

In [139]:
fig = returns.vbt.heatmap(
    x_level='comb_entry',
    y_level='comb_exit',
    slider_level='symbol',
)
fig.show()

## Create Volume Graph with Multiple Parameters

In [140]:
#create indicator object so Vectorbt can recognize it for any custom indicators
indicator2 = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'ma_window', 'entry', 'exit'],
    output_names=['value']
).from_apply_func(custom_indicator2, rsi_window=14, ma_window=50, entry=30, exit=70, keep_pd=True)

#run and get indicator values
res = indicator2.run(btc_price, rsi_window=np.arange(2,10,step=1, dtype=int),
                     #ma_window=np.arange(20,200,step=15, dtype=int),
                     entry=np.arange(20,30,step=1, dtype=int),
                     exit=np.arange(70,85,step=1, dtype=int), param_product=True) #param_product = True shows a combination of all the parameters

entries = res.value == 1.0
exits = res.value == -1.0

# https://vectorbt.dev/api/portfolio/base/#custom-metrics
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, seed=42, freq='m')
returns = pf.total_return()#print out everything by using .to_string()
print(returns.max(), returns.idxmax()) #print out the max combination of returns

fig = returns.vbt.volume(
    x_level='comb_entry',
    y_level='comb_exit',
    z_level='comb_rsi_window',
    slider_level='symbol',
)

fig.show()

0.02715492151088597 (8, 22, 70, 'BTC-USD')


# 3. Optimization Techniques to Speedup Backtesting
* Use TA-Lib directly as it is programmed to run fast with C with vbt wrapper
* Use @njit to convert to machine code for faster compilation
* Run parameter lookup in for loops to reduce RAM consumption but may slow down speed in general




In [141]:
# #Google Colab runs on Linux environment, so download TA-Lib C library first
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzvf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib
# !./configure --prefix=/usr
# !make
# !make install
# %cd ..
# !rm -rf ta-lib ta-lib-0.4.0-src.tar.gz

# #Then install the Python wrapper using pip
# !pip install TA-Lib

In [142]:
import pandas as pd
import talib
import datetime
#!pip3 install numba
from numba import njit

#copied code from above to optimize
end_time = datetime.datetime.now()
start_time= end_time - datetime.timedelta(days=3)

#save btc_price into csv form for faster processing, can comment out YFData download before optimization run
btc_price = vbt.YFData.download(['BTC-USD', 'ETH-USD'], missing_index='drop', start= start_time, end=end_time, interval='1m').get('Close')
btc_price.to_csv('data.csv')
btc_price = pd.read_csv('data.csv')
btc_price['Datetime'] = pd.to_datetime(btc_price['Datetime'])
btc_price.set_index('Datetime', inplace=True)
print(btc_price)

# #narrow down to the BTC-USD column, only if we use TA-Lib directly without the vbt wrapper
# btc_price = btc_price['BTC-USD']
# print(btc_price)

#can use a vbt wrapper with TA-Lib
RSI = vbt.IndicatorFactory.from_talib('RSI')

#njit converts to machine code and compile to speed up compilation
@njit
def produce_signal(rsi, entry, exit):
  trend = np.where(rsi>exit, -1, 0)
  trend = np.where((rsi<entry), 1, trend)
  return trend


#revised and took out close_5m and ma to help make the example easier to run
def custom_indicator2(close, rsi_window=14, entry = 30, exit=70):
  #replace with talib function wrapper from vbt, this should run a bit quicker
  #rsi = vbt.RSI.run(close, window=rsi_window).rsi
  rsi = RSI.run(close,rsi_window).real.to_numpy()  #with vbt, need to call what needs to be extracted. convert to numpy to utilize njit
  return produce_signal(rsi, entry, exit)

#Removed all ma related items
indicator2 = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'entry', 'exit'],
    output_names=['value']
).from_apply_func(custom_indicator2, rsi_window=14, entry=30, exit=70, to_2d=False) #setting to_2d to false before using TA-LIB since TA-LIB cant process 2 columns

rsi_windows= np.arange(10,40, step=1, dtype=int)
entry_windows = np.arange(10,40, step=3, dtype=int)
exit_windows = np.arange(60,85, step=3, dtype=int)

#master list
master_returns = []

for window in rsi_windows:
  res = indicator2.run(btc_price, rsi_window=window, entry=entry_windows,exit=exit_windows, param_product=True) #param_product = True shows a combination of all the parameters

  entries = res.value == 1.0
  exits = res.value == -1.0
  # https://vectorbt.dev/api/portfolio/base/#custom-metrics
  pf = vbt.Portfolio.from_signals(btc_price, entries, exits, seed=42, freq='m')
  master_returns.append(pf.total_return())

print(master_returns)
returns = pd.concat(master_returns, axis=1) #concate all the series together
returns.columns = rsi_windows

print(returns.max(), returns.idxmax()) #print out the max combination of returns
#print(pf.stats()) #if vectorbt could not parse the frequency of the close, it will not return any duration in time units or metrics that requires annualization and throw a bunch of warnings

/usr/local/lib/python3.10/dist-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Dropping missing data points.



                                BTC-USD      ETH-USD
Datetime                                            
2023-12-18 23:41:00+00:00  42590.000000  2212.980957
2023-12-18 23:42:00+00:00  42605.515625  2213.933105
2023-12-18 23:43:00+00:00  42625.316406  2214.437012
2023-12-18 23:44:00+00:00  42619.828125  2214.601318
2023-12-18 23:45:00+00:00  42619.667969  2214.354004
...                                 ...          ...
2023-12-21 23:33:00+00:00  43870.878906  2234.244629
2023-12-21 23:34:00+00:00  43855.898438  2234.023438
2023-12-21 23:35:00+00:00  43870.332031  2237.512695
2023-12-21 23:37:00+00:00  43868.261719  2237.499512
2023-12-21 23:38:00+00:00  43853.152344  2236.379639

[3962 rows x 2 columns]
[comb_rsi_window  comb_entry  comb_exit         
10               10          60         BTC-USD   -0.002280
                                        ETH-USD   -0.007144
                             63         BTC-USD   -0.002166
                                        ETH-USD   -0.0034

# 4. Graphing/Dashboarding
* Scatterplots

In [143]:
import vectorbt as vbt
import datetime

end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=3)

btc_price = vbt.YFData.download(['BTC-USD'], missing_index='drop', start= start_date, end=end_date, interval='1m').get('Close')

fast_ma = vbt.MA.run(btc_price, 5)
slow_ma = vbt.MA.run(btc_price, 13)

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(btc_price, entries, exits, seed=42, freq='m')

print(pf.total_return())
pf.plot().show()

0.05174453902643521


In [144]:
pf.trades.plot_pnl().show()

# 5. Order Types

# 6. Avoid Over-fitting